In [6]:
import ifcopenshell
import csv
import re

def filter_spaces_by_name(spaces):
    """Filter IfcSpaces whose names consist solely of digits."""
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")

# Name of the target storey
storey_name = "Plan 10"

# Find the target storey
target_storey = None
for storey in ifc_file.by_type("IfcBuildingStorey"):
    if storey.Name == storey_name:
        target_storey = storey
        break

if not target_storey:
    print(f"Storey with name '{storey_name}' was not found.")
else:
    print(f"Storey '{storey_name}' found. OID: {target_storey.id()}")

    # Initialize a list to store the results
    space_to_walls = []

    # Find all IfcSpaces in the target storey and filter them
    all_spaces = []
    for space in ifc_file.by_type("IfcSpace"):
        if space.Decomposes:
            for rel in space.Decomposes:
                if rel.is_a("IfcRelAggregates") and rel.RelatingObject == target_storey:
                    all_spaces.append(space)

    filtered_spaces = filter_spaces_by_name(all_spaces)

    # Process each filtered space
    for space in filtered_spaces:
        space_guid = space.GlobalId
        print(f"Space found: {space.Name}, GUID: {space_guid}")

        # Find the IfcRelSpaceBoundary relationships
        walls = []
        for rel_space_boundary in ifc_file.by_type("IfcRelSpaceBoundary"):
            if rel_space_boundary.RelatingSpace == space:
                if rel_space_boundary.RelatedBuildingElement and rel_space_boundary.RelatedBuildingElement.is_a("IfcWall"):
                    wall_guid = rel_space_boundary.RelatedBuildingElement.GlobalId
                    walls.append(wall_guid)
                    print(f"  Wall GUID: {wall_guid}")

        # Append the collected data to the list
        space_to_walls.append({
            "space_name": space.Name,
            "space_guid": space_guid,
            "walls": walls
        })

    # Write the results to a CSV file
    with open('Output06_Spaces_Walls.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=';')
        csvwriter.writerow(["Space Name", "Space GUID", "Wall GUIDs"])
        for entry in space_to_walls:
            csvwriter.writerow([
                entry["space_name"],
                entry["space_guid"],
                ",".join(entry["walls"])
            ])

    print("Results have been saved to Output_Spaces_Walls.csv")


Storey 'Plan 10' found. OID: 118
Space found: 19, GUID: 0uSXiavKjD892bcKOjMA$1
  Wall GUID: 0aNC3YsFzEZf8uB9g1nShB
  Wall GUID: 0MhF3zuEzBXwC_J$8wDFXT
  Wall GUID: 0aNC3YsFzEZf8uB9g1nShB
  Wall GUID: 0MhF3zuEzBXwC_J$8wDFXT
Space found: 20, GUID: 0uSXiavKjD892bcKOjMA$4
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSHQ
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSHQ
Space found: 21, GUID: 0uSXiavKjD892bcKOjMA$x
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSHR
  Wall GUID: 0MhF3zuEzBXwC_J$8wDFXU
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSHR
  Wall GUID: 0MhF3zuEzBXwC_J$8wDFXU
Space found: 27, GUID: 0uSXiavKjD892bcKOjMA$f
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSeq
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSer
  Wall GUID: 0MhF3zuEzBXwC_J$8wDDMj
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSer
  Wall GUID: 0MhF3zuEzBXwC_J$8wDDMj
Space found: 28, GUID: 0uSXiavKjD892bcKOjMA$i
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSet
  Wall GUID: 0MhF3zuEzBXwC_J$8wDDII
  Wall GUID: 0aNC3YsFzEZf8uB9g1nSet
  Wall GUID: 0MhF3zuEzBXwC_J$8wDDII
Space found: 98, GUID: 2Im_2tj0r41BcHdJK8Kcz3
  Wall 